# Solution of Two-Point Boundary Value Problem using Shooting Method

## Working Principle:
The Shooting Method is a numerical technique to solve two-point boundary value problems (BVPs) by converting the BVP into an initial value problem (IVP) and iteratively solving it.

## Boundary Value Problem (BVP)
A second-order ODE:
**d²y/dx² = f(x, y, y')**

with boundary conditions:
**y(a) = α, y(b) = β**

## Approach
1. **Convert the second-order ODE into a system of two first-order ODEs:**
   - dy₁/dx = y₂
   - dy₂/dx = f(x, y₁, y₂)
   
   where y₁ = y and y₂ = dy/dx

2. **Solve this system** using an initial guess for y'(a) = y₂(a) (denoted as s)

3. **Use numerical integration** (e.g., Runge-Kutta) to compute y(b) for the guessed s

4. **Adjust s iteratively** (e.g., using Newton's method or the secant method) to ensure the computed y(b) matches the boundary condition y(b) = β

## Pseudocode

### Input:
- Define the ODE as d²y/dx² = f(x, y, y')
- Specify boundary conditions y(a) = α, y(b) = β
- Set the initial guess for s = y'(a)

### Convert to a system of first-order ODEs:
- y₁' = y₂
- y₂' = f(x, y₁, y₂)

### Iterative Procedure:
1. **Solve the system** using RK4 or other numerical methods for the current guess of s
2. **Compute the value of y(b)**
3. **Compare y(b) with the target boundary condition β:**
   - If |y(b) - β| is close to tolerance, stop
   - Otherwise, update s using: **s_new = s_old - (y(b) - β)/Slope_at_s**
   - (e.g., use the secant method to adjust s)

### Output:
- y(x) values that satisfy the BVP

In [2]:
import math

def rk4_system(functions, x0, y0_list, h, x_end):
    """RK4 method for system of ODEs"""
    n = len(functions)
    x_values = [x0]
    y_values = [[y0_list[i]] for i in range(n)]
    
    x = x0
    y = y0_list[:]
    
    while x < x_end:
        k1 = [h * functions[i](x, y) for i in range(n)]
        
        y_temp = [y[i] + k1[i]/2 for i in range(n)]
        k2 = [h * functions[i](x + h/2, y_temp) for i in range(n)]
        
        y_temp = [y[i] + k2[i]/2 for i in range(n)]
        k3 = [h * functions[i](x + h/2, y_temp) for i in range(n)]
        
        y_temp = [y[i] + k3[i] for i in range(n)]
        k4 = [h * functions[i](x + h, y_temp) for i in range(n)]
        
        for i in range(n):
            y[i] = y[i] + (1/6) * (k1[i] + 2*k2[i] + 2*k3[i] + k4[i])
        
        x = x + h
        x_values.append(x)
        for i in range(n):
            y_values[i].append(y[i])
    
    return x_values, y_values

def shooting_method(f, a, b, alpha, beta, s_guess=0.0, h=0.01, tolerance=1e-6, max_iterations=50):
    """
    Solve BVP using shooting method
    
    Parameters:
    f: function representing d²y/dx² = f(x, y, dy/dx)
    a, b: boundary points
    alpha: y(a) = alpha
    beta: y(b) = beta
    s_guess: initial guess for y'(a)
    h: step size for RK4
    tolerance: convergence tolerance
    max_iterations: maximum number of iterations
    
    Returns:
    x_values, y_values: solution arrays
    """
    
    def f1(x, vars):  # dy1/dx = y2
        return vars[1]
    
    def f2(x, vars):  # dy2/dx = f(x, y1, y2)
        return f(x, vars[0], vars[1])
    
    functions = [f1, f2]
    
    s1 = s_guess
    s2 = s_guess + 0.1  # Second guess for secant method
    
    print(f"Solving BVP: y({a}) = {alpha}, y({b}) = {beta}")
    print(f"Iteration\tGuess s\t\ty(b)\t\tError")
    print("-" * 50)
    
    for iteration in range(max_iterations):
        # Solve with current guess s1
        x_vals, y_vals = rk4_system(functions, a, [alpha, s1], h, b)
        y_b = y_vals[0][-1]  # y value at x = b
        error1 = y_b - beta
        
        print(f"{iteration+1}\t\t{s1:.6f}\t{y_b:.6f}\t{error1:.6f}")
        
        # Check convergence
        if abs(error1) < tolerance:
            print(f"Converged after {iteration+1} iterations!")
            return x_vals, y_vals[0]
        
        # If first iteration, compute error for s2
        if iteration == 0:
            x_vals2, y_vals2 = rk4_system(functions, a, [alpha, s2], h, b)
            y_b2 = y_vals2[0][-1]
            error2 = y_b2 - beta
        
        # Update using secant method
        if iteration > 0:
            s_new = s1 - error1 * (s1 - s2) / (error1 - error2)
            s2 = s1
            error2 = error1
        else:
            s_new = s2
        
        s1 = s_new
    
    print(f"Did not converge after {max_iterations} iterations")
    return x_vals, y_vals[0]

# Example 1: y'' + y = 0, y(0) = 1, y(π) = -1
print("Example 1: y'' + y = 0, y(0) = 1, y(π) = -1")
def f1(x, y, dy_dx):
    return -y

a1, b1 = 0, math.pi
alpha1, beta1 = 1, -1

x_vals1, y_vals1 = shooting_method(f1, a1, b1, alpha1, beta1, s_guess=0.0, h=0.01)

# Exact solution: y = cos(x)
print("\nSolution comparison:")
print("x\t\tNumerical\tExact\t\tError")
print("-" * 50)
for i in range(0, len(x_vals1), len(x_vals1)//10):
    x = x_vals1[i]
    y_num = y_vals1[i]
    y_exact = math.cos(x)
    error = abs(y_exact - y_num)
    print(f"{x:.4f}\t\t{y_num:.6f}\t{y_exact:.6f}\t{error:.8f}")

print("\n" + "="*70 + "\n")

# Example 2: y'' - 2y' + y = x*e^x, y(0) = 0, y(1) = 0
print("Example 2: y'' - 2y' + y = x*e^x, y(0) = 0, y(1) = 0")
def f2(x, y, dy_dx):
    return 2*dy_dx - y + x*math.exp(x)

a2, b2 = 0, 1
alpha2, beta2 = 0, 0

x_vals2, y_vals2 = shooting_method(f2, a2, b2, alpha2, beta2, s_guess=1.0, h=0.01)

print("\nSolution:")
print("x\t\ty(x)")
print("-" * 20)
for i in range(0, len(x_vals2), len(x_vals2)//10):
    x = x_vals2[i]
    y = y_vals2[i]
    print(f"{x:.4f}\t\t{y:.6f}")

print("\n" + "="*70 + "\n")

# Example 3: y'' + 4y = 4x, y(0) = 0, y(1) = 2
print("Example 3: y'' + 4y = 4x, y(0) = 0, y(1) = 2")
def f3(x, y, dy_dx):
    return -4*y + 4*x

a3, b3 = 0, 1
alpha3, beta3 = 0, 2

x_vals3, y_vals3 = shooting_method(f3, a3, b3, alpha3, beta3, s_guess=2.0, h=0.01)

# Exact solution: y = x + A*sin(2x) + B*cos(2x)
# With boundary conditions: y = x + sin(2x)
print("\nSolution comparison:")
print("x\t\tNumerical\tExact\t\tError")
print("-" * 50)
for i in range(0, len(x_vals3), len(x_vals3)//10):
    x = x_vals3[i]
    y_num = y_vals3[i]
    y_exact = x + math.sin(2*x)
    error = abs(y_exact - y_num)
    print(f"{x:.4f}\t\t{y_num:.6f}\t{y_exact:.6f}\t{error:.8f}")

print("\n" + "="*70 + "\n")

# Example 4: Nonlinear BVP - y'' + y^2 = 0, y(0) = 1, y(1) = 0.5
print("Example 4: Nonlinear BVP - y'' + y² = 0, y(0) = 1, y(1) = 0.5")
def f4(x, y, dy_dx):
    return -y**2

a4, b4 = 0, 1
alpha4, beta4 = 1, 0.5

x_vals4, y_vals4 = shooting_method(f4, a4, b4, alpha4, beta4, s_guess=-1.0, h=0.01, tolerance=1e-5)

print("\nSolution:")
print("x\t\ty(x)")
print("-" * 20)
for i in range(0, len(x_vals4), len(x_vals4)//10):
    x = x_vals4[i]
    y = y_vals4[i]
    print(f"{x:.4f}\t\t{y:.6f}")

print("\n" + "="*70 + "\n")

# Example 5: y'' - xy' + y = 0, y(0) = 1, y(2) = 3
print("Example 5: y'' - xy' + y = 0, y(0) = 1, y(2) = 3")
def f5(x, y, dy_dx):
    return x*dy_dx - y

a5, b5 = 0, 2
alpha5, beta5 = 1, 3

x_vals5, y_vals5 = shooting_method(f5, a5, b5, alpha5, beta5, s_guess=1.0, h=0.01)

print("\nSolution:")
print("x\t\ty(x)\t\ty'(x)")
print("-" * 30)
# Also compute y'(x) for this example
def f5_1(x, vars):
    return vars[1]
def f5_2(x, vars):
    return x*vars[1] - vars[0]

functions5 = [f5_1, f5_2]
# Use the final slope found by shooting method
final_slope = (y_vals5[1] - y_vals5[0]) / (x_vals5[1] - x_vals5[0])  # approximate
x_final, y_final = rk4_system(functions5, a5, [alpha5, final_slope], 0.01, b5)

for i in range(0, len(x_final), len(x_final)//10):
    x = x_final[i]
    y = y_final[0][i]
    dy = y_final[1][i]
    print(f"{x:.4f}\t\t{y:.6f}\t{dy:.6f}")

Example 1: y'' + y = 0, y(0) = 1, y(π) = -1
Solving BVP: y(0) = 1, y(3.141592653589793) = -1
Iteration	Guess s		y(b)		Error
--------------------------------------------------
1		0.000000	-0.999965	0.000035
2		0.100000	-1.000805	-0.000805


ZeroDivisionError: float division by zero